In [3]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

In [4]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()



In [5]:
s3 = boto3.resource('s3')
b = s3.Bucket('sagemaker-mlai-harvesting')
!mkdir data
b.download_file( 'data/MLAI_ParsedDataSet.tsv', 'data/data.tsv')

# !head data/data.tsv

In [28]:
txn = pd.read_csv('data/data.tsv',sep='\t')
txn.head()

,Act,Activity,Log_Time,CustID,CustName,MarketID,CustCountry,CustState,GroupID,ProfID,...,Search_Term,Num_of_Results,DbName,AN,Retrieval_Format,Retrieval_Mode,Checkout_Result,Checkout_Type,Violation_Type,Bad_Actor
0,121,Search Click,2019-05-01 19:00:00.000,s8989762,COLUMBIA COLLEGE,academic,UNITED STATES,ILLINOIS,main,ehost,...,new jobs new inequalities,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,115,Retrievals,2019-05-01 19:00:00.000,s8983751,UNIV OF LA VERNE,academic,UNITED STATES,CALIFORNIA,main,ehost,...,NaN,NaN,aph,96191495,Abstract,View,NaN,NaN,NaN,0
2,115,Retrievals,2019-05-01 19:00:01.000,s8969716,SHENANDOAH UNIVERSITY,academic,UNITED STATES,VIRGINIA,main,ehost,...,NaN,NaN,e000xna,171851,Abstract,View,NaN,NaN,NaN,0
3,111,Session Start,2019-05-01 19:00:01.000,s8987536,FLORIDA A & M UNIV,academic,UNITED STATES,FLORIDA,main,wsapi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,121,Search Click,2019-05-01 19:00:01.000,s8987536,FLORIDA A & M UNIV,academic,UNITED STATES,FLORIDA,main,wsapi,...,mass incarceration,17533.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [190]:
txn.columns

Index(['Act', 'Activity', 'Log_Time', 'CustID', 'CustName', 'MarketID',
       'CustCountry', 'CustState', 'GroupID', 'ProfID', 'Int_ID', 'Client_IP',
       'UserID', 'CookieGUID', 'SessionNo', 'UserAgent', 'Login_Method',
       'Personalization', 'Search_From', 'EP_Db_Count', 'Search_Term',
       'Num_of_Results', 'DbName', 'AN', 'Retrieval_Format', 'Retrieval_Mode',
       'Checkout_Result', 'Checkout_Type', 'Violation_Type', 'Bad_Actor'],
      dtype='object')

In [243]:
txns = np.sort(txn['Act'].unique())
txni= pd.DataFrame( txns )
txni.columns = ['txn']
# txni['txni']= txni.index
# txni.set_index('txn')

txni


,txn
0,111
1,112
2,114
3,115
4,116
5,117
6,118
7,119
8,121
9,123


In [319]:
def flatten_txns( txn_log, txn_types ):
    txn_narrow = txn_log[['SessionNo', 'Act','Bad_Actor']]
    txn_pivot = pd.pivot_table(txn_narrow, index=['SessionNo','Bad_Actor'], columns = ['Act'],aggfunc=[np.size]).fillna(0)
    txn_pivot.columns = txn_pivot.columns.droplevel(0)
    txn_flat = txn_pivot.rename_axis(None, axis=1).reset_index()
    return txn_flat

In [318]:
flatten_txns( txn, txni ).head(10)

,SessionNo,Bad_Actor,111,112,114,115,116,117,118,119,...,402,403,406,407,410,411,511,513,601,607
0,-2147317281,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-2147002735,0,3.0,0.0,0.0,6.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-2146953899,0,0.0,3.0,0.0,60.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-2146926264,0,3.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-2146915841,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-2146723372,0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-2146089473,0,3.0,0.0,0.0,3.0,3.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-2145083328,0,3.0,0.0,0.0,9.0,3.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,-2145021172,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,-2144988764,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [314]:
flat = flatten_txns( txn, txni )

In [267]:
# def func_split( flat ):
#     flat_x = shuffle.drop('Bad_Actor', axis=1)
#     flat_y = shuffle['Bad_Actor']
#     return flat_x, flat_y

def split_frame( df, train_frac ):
    l = len(df)
    test_frac = (1-train_frac)/2
    tr = int(train_frac * l)
    te = int(tr + test_frac * l)
    
    train = df[:tr]
    test = df[tr:te]
    val = df[te:]
    return [train, test, val]

In [283]:
sets= split_frame(np.arange(10),.4)
df = []
for i in sets:
    np.random.shuffle(i)
    df.append( i)
    
df

[array([2, 0, 1, 3]), array([4, 5, 6]), array([9, 8, 7])]

In [327]:
flat[flat['Bad_Actor'] == 0]

,SessionNo,Bad_Actor,111,112,114,115,116,117,118,119,...,402,403,406,407,410,411,511,513,601,607
0,-2147317281,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-2147002735,0,3.0,0.0,0.0,6.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-2146953899,0,0.0,3.0,0.0,60.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-2146926264,0,3.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-2146915841,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-2146723372,0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-2146089473,0,3.0,0.0,0.0,3.0,3.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-2145083328,0,3.0,0.0,0.0,9.0,3.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,-2145021172,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,-2144988764,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [328]:
def train_split( flat, bad_split=.8 ):
    bad = flat[flat['Bad_Actor']==1]
    good = flat[flat['Bad_Actor']==0]
    
    bads = split_frame(bad, bad_split)
    goods = split_frame(good, bad_split)
    
    dfs = []
    for i in range(2):
        df = pd.merge( bads[i], goods[i])
        dfs.append( df )
    
    return [bads,goods]
    
#     for i in range(2):
#         xs
    
#     flat_x = shuffle.drop('Bad_Actor', axis=1)
#     flat_y = shuffle['Bad_Actor']

#     shuf = flat.sample(frac=1).reset_index(drop=True)
    
#     return train_x, train_y, test_x, test_y, val_x, val_y
    

In [329]:
dfs = train_split(flat)

    
dfs[1]


[        SessionNo  Bad_Actor  111  112  114   115   116  117  118  119  ...  \
 0     -2147317281          0  3.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  ...   
 1     -2147002735          0  3.0  0.0  0.0   6.0   3.0  0.0  0.0  0.0  ...   
 2     -2146953899          0  0.0  3.0  0.0  60.0   0.0  0.0  0.0  0.0  ...   
 3     -2146926264          0  3.0  0.0  0.0   3.0   0.0  3.0  0.0  0.0  ...   
 4     -2146915841          0  3.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  ...   
 5     -2146723372          0  3.0  0.0  0.0   0.0   0.0  3.0  0.0  0.0  ...   
 6     -2146089473          0  3.0  0.0  0.0   3.0   3.0  3.0  0.0  0.0  ...   
 7     -2145083328          0  3.0  0.0  0.0   9.0   3.0  3.0  0.0  0.0  ...   
 8     -2145021172          0  0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  ...   
 9     -2144988764          0  0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  ...   
 10    -2143861029          0  3.0  0.0  0.0   6.0   0.0  0.0  0.0  0.0  ...   
 11    -2143133687          0  3.0  0.0 